In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
import lightgbm as lgb
import datetime as dt
import matplotlib.dates as mdates
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error,r2_score
from math import sqrt

In [2]:
%%time
root = '/Users/yuyizhang/Documents/forecasting/Energy_Data/Forecasting Energy Consumption_data/'
holidays=pd.read_csv(root + 'holidays.csv')
metadata=pd.read_csv(root + 'metadata.csv')
train=pd.read_csv(root + 'train.csv')
weather=pd.read_csv(root + 'weather.csv')

CPU times: user 15.2 s, sys: 2.19 s, total: 17.4 s
Wall time: 17.6 s


In [3]:
holidays = holidays.rename(columns={"Date": "Timestamp"})

In [4]:
#format datetime
holidays['Timestamp']= pd.to_datetime(holidays['Timestamp'],format='%d/%m/%Y')
train['Timestamp']= pd.to_datetime(train['Timestamp'],format='%Y-%m-%d %H:%M:%S')
weather['Timestamp']= pd.to_datetime(weather['Timestamp'],format='%Y-%m-%d %H:%M:%S')

In [5]:
train

,obs_id,SiteId,Timestamp,ForecastId,Value
0,744519,1,2014-09-03 00:00:00,1,9.096555e+05
1,7627564,1,2014-09-04 00:00:00,1,1.748273e+06
2,7034705,1,2014-09-05 00:00:00,1,NaN
3,5995486,1,2014-09-06 00:00:00,1,NaN
4,7326510,1,2014-09-07 00:00:00,1,NaN
...,...,...,...,...,...
6559825,2344172,305,2015-11-19 11:30:00,6974,3.824641e+03
6559826,5113853,305,2015-11-19 11:45:00,6974,3.766131e+03
6559827,5227641,305,2015-11-19 12:00:00,6974,3.736505e+03
6559828,3750622,305,2015-11-19 12:15:00,6974,3.816494e+03


In [6]:
merge_df = pd.merge(train,metadata,how = 'outer',on = ['SiteId'])
merge_df = merge_df.dropna(axis = 0, how = 'any')

In [7]:
del train, metadata

In [8]:
# Below function extracts date related features from datetime
def create_date_featues_train(df):

    df['Year'] = pd.to_datetime(merge_df['Timestamp']).dt.year
    df['Month'] = pd.to_datetime(merge_df['Timestamp']).dt.month
    df['Day'] = pd.to_datetime(merge_df['Timestamp']).dt.day
    df['Hour'] = pd.to_datetime(merge_df['Timestamp']).dt.hour
    df['Minute'] = pd.to_datetime(merge_df['Timestamp']).dt.minute
    
    return df

In [9]:
merge_df = create_date_featues_train(merge_df)

In [10]:
merge_df.shape

(6473229, 20)

In [11]:
merge_df.columns

Index(['obs_id', 'SiteId', 'Timestamp', 'ForecastId', 'Value', 'Surface',
       'Sampling', 'BaseTemperature', 'MondayIsDayOff', 'TuesdayIsDayOff',
       'WednesdayIsDayOff', 'ThursdayIsDayOff', 'FridayIsDayOff',
       'SaturdayIsDayOff', 'SundayIsDayOff', 'Year', 'Month', 'Day', 'Hour',
       'Minute'],
      dtype='object')

In [12]:
df_train = pd.merge(merge_df,holidays,how = 'outer',on = ['Timestamp'])

In [13]:
df_train['Holiday'] = df_train['Holiday'].replace(np.nan, 0)
df_train = df_train.dropna(axis = 0, how = 'any')

In [14]:
for col in ['Holiday']:
    df_train = pd.get_dummies(df_train, columns=[col])
for col in ['Sampling']:
    merge_df = pd.get_dummies(merge_df, columns=[col])
for col in ['BaseTemperature']:
    merge_df = pd.get_dummies(merge_df, columns=[col])
for col in ['MondayIsDayOff']:
    merge_df = pd.get_dummies(merge_df, columns=[col])
for col in ['TuesdayIsDayOff']:
    merge_df = pd.get_dummies(merge_df, columns=[col]) 
for col in ['WednesdayIsDayOff']:
    merge_df = pd.get_dummies(merge_df, columns=[col])
for col in ['ThursdayIsDayOff']:
    merge_df = pd.get_dummies(merge_df, columns=[col])
for col in ['FridayIsDayOff']:
    merge_df = pd.get_dummies(merge_df, columns=[col]) 
for col in ['SaturdayIsDayOff']:
    merge_df = pd.get_dummies(merge_df, columns=[col])
for col in ['SundayIsDayOff']:
    merge_df = pd.get_dummies(merge_df, columns=[col]) 
print(merge_df.shape)

(6473229, 28)


In [15]:
data_reset = merge_df[merge_df['Timestamp']<'2013-10-30 00:00']

In [16]:
data_reset 

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Year,Month,Day,Hour,...,MondayIsDayOff_False,TuesdayIsDayOff_False,WednesdayIsDayOff_False,ThursdayIsDayOff_False,FridayIsDayOff_False,FridayIsDayOff_True,SaturdayIsDayOff_False,SaturdayIsDayOff_True,SundayIsDayOff_False,SundayIsDayOff_True
900,4382312,2,2013-01-01 01:00:00,5,3.015996e+04,6098.278376,2013,1,1,1,...,1,1,1,1,1,0,0,1,0,1
901,2016541,2,2013-01-01 02:00:00,5,2.979354e+04,6098.278376,2013,1,1,2,...,1,1,1,1,1,0,0,1,0,1
902,78869,2,2013-01-01 03:00:00,5,3.168241e+04,6098.278376,2013,1,1,3,...,1,1,1,1,1,0,0,1,0,1
903,1361014,2,2013-01-01 04:00:00,5,2.988183e+04,6098.278376,2013,1,1,4,...,1,1,1,1,1,0,0,1,0,1
904,129169,2,2013-01-01 05:00:00,5,3.056033e+04,6098.278376,2013,1,1,5,...,1,1,1,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6557319,5653279,303,2013-10-25 00:00:00,6966,1.054313e+07,17934.445927,2013,10,25,0,...,1,1,1,1,1,0,0,1,0,1
6557320,7663267,303,2013-10-26 00:00:00,6966,1.054313e+07,17934.445927,2013,10,26,0,...,1,1,1,1,1,0,0,1,0,1
6557321,5767884,303,2013-10-27 00:00:00,6966,1.054313e+07,17934.445927,2013,10,27,0,...,1,1,1,1,1,0,0,1,0,1
6557322,1631678,303,2013-10-28 00:00:00,6966,1.054313e+07,17934.445927,2013,10,28,0,...,1,1,1,1,1,0,0,1,0,1


In [17]:
train_1 = data_reset[data_reset['Timestamp']<'2013-08-15 00:00']
test_1 = data_reset[data_reset['Timestamp']>='2013-08-15 00:00']

In [18]:
print(train_1.shape)
print(test_1.shape)

(271803, 28)
(78380, 28)


In [19]:
x_train1 = train_1.drop(columns={'Timestamp','Value'},axis=1)
y_train1 = train_1.loc[:,['Value']]

x_val1=test_1.drop(columns={'Timestamp','Value'},axis=1)
y_val1=test_1.loc[:,['Value']]

In [20]:
x_train1.head()

,obs_id,SiteId,ForecastId,Surface,Year,Month,Day,Hour,Minute,Sampling_5.0,...,MondayIsDayOff_False,TuesdayIsDayOff_False,WednesdayIsDayOff_False,ThursdayIsDayOff_False,FridayIsDayOff_False,FridayIsDayOff_True,SaturdayIsDayOff_False,SaturdayIsDayOff_True,SundayIsDayOff_False,SundayIsDayOff_True
900,4382312,2,5,6098.278376,2013,1,1,1,0,0,...,1,1,1,1,1,0,0,1,0,1
901,2016541,2,5,6098.278376,2013,1,1,2,0,0,...,1,1,1,1,1,0,0,1,0,1
902,78869,2,5,6098.278376,2013,1,1,3,0,0,...,1,1,1,1,1,0,0,1,0,1
903,1361014,2,5,6098.278376,2013,1,1,4,0,0,...,1,1,1,1,1,0,0,1,0,1
904,129169,2,5,6098.278376,2013,1,1,5,0,0,...,1,1,1,1,1,0,0,1,0,1


In [28]:
from sklearn.preprocessing import MinMaxScaler

In [29]:
train_a = x_train1.copy()
train_b = y_train1.copy()

test_c = x_val1.copy()
test_d = y_val1.copy()

In [30]:
scaler = MinMaxScaler()
x_train_trans = scaler.fit_transform(train_a)
test_trans = scaler.transform(test_c)

In [31]:
y_train_trans = scaler.fit_transform(train_b)
y_val_trans =scaler.fit_transform(test_d)

In [32]:
x_train_va = np.array(x_train_trans)
y_train_va = np.array(y_train_trans)

x_val_va = np.array(test_trans)
y_val_va = np.array(y_val_trans)

In [33]:
x_train = np.array(x_train1)
y_train = np.array(y_train1)

x_val = np.array(x_val1)
y_val = np.array(y_val1)

In [34]:
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1,
                }

In [35]:
lgb_train = lgb.Dataset(x_train_va, y_train_va)
lgb_eval = lgb.Dataset(x_val_va, y_val_va)

In [36]:
%%time

evals_result = {}

model_DC = lgb.train(lgb_params, lgb_train,
                      valid_sets=[lgb_eval], 
                      verbose_eval=100, 
                      evals_result=evals_result)

[100]	valid_0's rmse: 0.197491
[200]	valid_0's rmse: 0.0604429
[300]	valid_0's rmse: 0.0313308
[400]	valid_0's rmse: 0.0234372
[500]	valid_0's rmse: 0.0192546
[600]	valid_0's rmse: 0.0153104
[700]	valid_0's rmse: 0.0132313
[800]	valid_0's rmse: 0.0124625
[900]	valid_0's rmse: 0.0120646
[1000]	valid_0's rmse: 0.0118984
[1100]	valid_0's rmse: 0.0118621
[1200]	valid_0's rmse: 0.0117947
[1300]	valid_0's rmse: 0.0117372
[1400]	valid_0's rmse: 0.0117469
[1500]	valid_0's rmse: 0.0117131
[1600]	valid_0's rmse: 0.0116934
[1700]	valid_0's rmse: 0.0116951
[1800]	valid_0's rmse: 0.0116343
[1900]	valid_0's rmse: 0.011632
[2000]	valid_0's rmse: 0.0116105
[2100]	valid_0's rmse: 0.0115992
[2200]	valid_0's rmse: 0.0116227
[2300]	valid_0's rmse: 0.0115979
[2400]	valid_0's rmse: 0.0116134
[2500]	valid_0's rmse: 0.0115616
[2600]	valid_0's rmse: 0.0115741
[2700]	valid_0's rmse: 0.0116405
[2800]	valid_0's rmse: 0.011639
[2900]	valid_0's rmse: 0.011665
[3000]	valid_0's rmse: 0.0116884
CPU times: user 4min 36

In [37]:
pred=model_DC.predict(x_val_va)
r2_score(y_val_va,pred)

0.9381656093081585

In [38]:
mean_squared_error(y_val_va,pred)

0.00013661759214495173

In [39]:
import sklearn.ensemble

In [40]:
%%time
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(x_train_va, y_train_va)

CPU times: user 1min 25s, sys: 678 ms, total: 1min 25s
Wall time: 1min 26s


RandomForestRegressor(random_state=0)

In [41]:
pred_rf = rf.predict(x_val_va)
r2_score(y_val_va,pred_rf)

0.9297208143240488

In [43]:
mean_squared_error(y_val_va,pred_rf)

0.0001552756163282329

In [35]:
x_train1 = x_train_trans.reshape(x_train_trans.shape[0],1,x_train_trans.shape[-1])
x_test1 = test_trans.reshape(test_trans.shape[0],1,test_trans.shape[-1])

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

from sklearn import preprocessing
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization,SimpleRNN,Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

from keras.layers import Convolution1D, MaxPool1D,Conv1D,MaxPooling1D

import tensorflow as tf 
import matplotlib.pyplot as plt 
from tensorflow.keras import Sequential 

from tensorflow.keras import datasets, layers, models,Input
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score 
from tensorflow.keras.layers import Dense,LSTM,Dropout,GRU,SimpleRNN,Bidirectional 
from sklearn import metrics 

Using TensorFlow backend.


In [38]:
%%time

np.random.seed(0)
tf.random.set_seed(0)

#LSTM model

model = tf.keras.Sequential([ 
 BatchNormalization(),
 Bidirectional(LSTM(128, return_sequences=True,activation='relu')),
#  SimpleRNN(64, return_sequences=True,activation='linear'),
 Dropout(0.2), 
 BatchNormalization(),
 Bidirectional(LSTM(128,return_sequences=False)),
 Dropout(0.2), 
 Dense(1) 
]) 

model.compile(loss='mse', optimizer='adam') 

# history = model.fit(x_train, y_train, batch_size=5260, epochs=1, validation_data=(x_test, y_test))
history = model.fit(x_train1, y_train_trans, batch_size=128, epochs=3,validation_data=(x_test1, y_val_trans)) 
# model.build(input_shape=(128,1,27))
model.summary()

Epoch 1/3
2124/2124 [==============================] - 24s 11ms/step - loss: 0.0031 - val_loss: 2.1788e-04
Epoch 2/3
2124/2124 [==============================] - 25s 12ms/step - loss: 6.1565e-04 - val_loss: 1.6546e-04
Epoch 3/3
2124/2124 [==============================] - 26s 12ms/step - loss: 4.8971e-04 - val_loss: 1.8867e-04
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch multiple                  104       
_________________________________________________________________
bidirectional_2 (Bidirection multiple                  158720    
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0         
_________________________________________________________________
batch_normalization_3 (Batch multiple                  1024      
_______________________________________________________

In [39]:
pred_lstm=model.predict(x_test1)

mean_squared_error(y_val_trans,pred_lstm)

0.00018867427305597117

In [41]:
r2_score(y_val_trans,pred_lstm)

0.914604272184333

In [65]:
%%time
np.random.seed(0)
tf.random.set_seed(0)

#RNN model

model_rnn = tf.keras.Sequential([ 
 BatchNormalization(),
 Bidirectional(SimpleRNN(128, return_sequences=True,activation='relu')),
#  SimpleRNN(64, return_sequences=True,activation='linear'),
 Dropout(0.2), 
 BatchNormalization(),
 Bidirectional(SimpleRNN(128,return_sequences=False)),
 Dropout(0.2), 
 Dense(1) 
]) 

model_rnn.compile(loss='mse', optimizer='adam') 

# history = model.fit(x_train, y_train, batch_size=5260, epochs=1, validation_data=(x_test, y_test))
history_rnn = model_rnn.fit(x_train1, y_train_trans, batch_size=128, epochs=5,validation_data=(x_test1, y_val_trans)) 
# model.build(input_shape=(128,1,27))
model_rnn.summary()

Epoch 1/5
2124/2124 [==============================] - 11s 5ms/step - loss: 0.0287 - val_loss: 0.0016
Epoch 2/5
2124/2124 [==============================] - 10s 5ms/step - loss: 0.0040 - val_loss: 0.0012
Epoch 3/5
2124/2124 [==============================] - 10s 5ms/step - loss: 0.0022 - val_loss: 6.6278e-04
Epoch 4/5
2124/2124 [==============================] - 11s 5ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 5/5
2124/2124 [==============================] - 11s 5ms/step - loss: 0.0013 - val_loss: 3.1031e-04
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_18 (Batc multiple                  104       
_________________________________________________________________
bidirectional_18 (Bidirectio multiple                  39680     
_________________________________________________________________
dropout_18 (Dropout)         multiple                  0        

In [66]:
pred_rnn=model_rnn.predict(x_test1)

mean_squared_error(y_val_trans,pred_rnn)

0.0003103134542792522

In [67]:
r2_score(y_val_trans,pred_rnn)

0.8595492493493841

In [68]:
%%time
np.random.seed(0)
tf.random.set_seed(0)

#GRU model

model_gru = tf.keras.Sequential([ 
 BatchNormalization(),
 Bidirectional(GRU(128, return_sequences=True,activation='relu')),
#  SimpleRNN(64, return_sequences=True,activation='linear'),
 Dropout(0.2), 
 BatchNormalization(),
 Bidirectional(GRU(128,return_sequences=False)),
 Dropout(0.2), 
 Dense(1) 
]) 

model_gru.compile(loss='mse', optimizer='adam') 

# history = model.fit(x_train, y_train, batch_size=5260, epochs=1, validation_data=(x_test, y_test))
history_gru = model_gru.fit(x_train1, y_train_trans, batch_size=128, epochs=5,validation_data=(x_test1, y_val_trans)) 
# model.build(input_shape=(128,1,27))
model_gru.summary()

Epoch 1/5
2124/2124 [==============================] - 20s 9ms/step - loss: 0.0081 - val_loss: 3.5713e-04
Epoch 2/5
2124/2124 [==============================] - 22s 10ms/step - loss: 9.0575e-04 - val_loss: 2.7037e-04
Epoch 3/5
2124/2124 [==============================] - 21s 10ms/step - loss: 6.6666e-04 - val_loss: 2.4510e-04
Epoch 4/5
2124/2124 [==============================] - 21s 10ms/step - loss: 4.8599e-04 - val_loss: 1.7282e-04
Epoch 5/5
2124/2124 [==============================] - 21s 10ms/step - loss: 4.5190e-04 - val_loss: 1.8429e-04
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_20 (Batc multiple                  104       
_________________________________________________________________
bidirectional_20 (Bidirectio multiple                  119808    
_________________________________________________________________
dropout_20 (Dropout)         mu

In [69]:
pred_gru=model_gru.predict(x_test1)

mean_squared_error(y_val_trans,pred_gru)

0.00018428657939782245

In [70]:
r2_score(y_val_trans,pred_gru)

0.9165901830735121